## Process a GeoJSON

Python Notebook pathway for [Whisp](https://openforis.org/solutions/whisp/) for running locally.

- Use this notebook for datasets up to around 100,000 features. For larger datasets consider the 'whisp_ee_asset_to_drive.ipynb' notebook.
- Please log any issues/requests [here](https://github.com/forestdatapartnership/whisp/issues)

Setup
- Use a [virtual environment](https://docs.python.org/3/tutorial/venv.html) to avoid altering your python environment 

**To run:** click play buttons (or press shift + enter) 

**Requirements:** Google Earth Engine (GEE) account and registered cloud project.

- **Aim:** support compliance with zero deforestation regulations
- **Input**: geojson file of plot boundaries or points
- **Output**: CSV table and geojson containing statistics and risk indicators



In [ ]:
# Earth Engine and Common Libraries
import ee
from pathlib import Path
import datetime

# Reset Earth Engine to clear any previous sessions
ee.Reset()
# Authenticate and initialize Earth Engine. 
try:
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')  # Try to use existing credentials first
except Exception:
    ee.Authenticate() # Authenticate may open a browser window
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

# NB if not working add your cloud project: ee.Initialize(project="your_gee_cloud_project_name",opt_url='https://earthengine-highvolume.googleapis.com')

In [ ]:
# Install openforis-whisp (uncomment line if not already installed)
# !pip install --pre openforis-whisp

# NB for editable mode install via your terminal with: pip install -e .[dev]

In [ ]:
import openforis_whisp as whisp

Get a geojson

In [ ]:
GEOJSON_EXAMPLE_FILEPATH = whisp.get_example_data_path("geojson_example.geojson")

Prepare layers

In [ ]:
# Choose if want to include additional custom layers
USE_CUSTOM_BANDS = False # set to True if want to add extra ee data to whisp

In [ ]:
# =============================================================================
# CUSTOM BANDS SETUP (OPTIONAL) - runs only if USE_CUSTOM_BANDS = True above
# =============================================================================
if USE_CUSTOM_BANDS:

    # Step 1: Define custom Earth Engine images (binary values 0 or 1). Currently showing example placeholders.
    custom_images = {
        'example_treecover': ee.Image(1),  # ee.Image("UMD/hansen/global_forest_change_2024_v1_12").select("treecover2000").gt(10).selfMask()
        'nXX_example_commodity': ee.Image(1) # ee.ImageCollection("projects/forestdatapartnership/assets/cocoa/model_2025a").filter(ee.Filter.date('2020-01-01', '2021-01-01')).mosaic().gt(.8).selfMask()
        # add more images as needed (prefix 'nXX_' = iso2 code for national dataset)
    }

    # Step 2: Define metadata for each custom band (keys must match above)
    # Themes: 'treecover', 'commodities', 'disturbance_before', 'disturbance_after'
    # Timber themes: 'primary', 'naturally_reg_2020', 'planted_plantation_2020', etc.
    custom_bands_info = {
        'example_treecover': {
            'ISO2_code': "",          # Country code (empty = all countries)
            'theme': 'treecover',     # Risk theme
            'theme_timber': "",       # Timber theme (if applicable)
            'use_for_risk': 1,        # Include in risk calculations (1=yes, 0=no)
            'use_for_risk_timber': 0  # Include in timber risk (1=yes, 0=no)
        },
        'nXX_example_commodity': {
            'ISO2_code': "XX", 
            'theme': 'commodities', 
            'theme_timber': "",
            'use_for_risk': 1, 
            'use_for_risk_timber': 0
        }
        # add more band metadata as needed
    }

    # Step 3: Combine custom bands and extract names
    custom_ee_image = whisp.combine_custom_bands(custom_images, custom_bands_info)

    custom_bands = list(custom_bands_info.keys())


In [ ]:
# Choose additional national datasets to include (currently four countries: 'co', 'ci', 'br', 'cm').
base_iso2_codes = ['co', 'ci', 'br', 'cm']

# automatically add any custom ISO2 codes from custom_bands_info if USE_CUSTOM_BANDS is True
iso2_codes_list = base_iso2_codes.copy()
if USE_CUSTOM_BANDS:
    iso2_codes_list += [code.lower() for code in {v.get('ISO2_code') for v in custom_bands_info.values()} if code and code.lower() not in iso2_codes_list]

In [ ]:
# Create final Whisp image
whisp_image = whisp.combine_datasets(national_codes=iso2_codes_list)
standard_bands = len(whisp_image.bandNames().getInfo())

if USE_CUSTOM_BANDS and 'custom_ee_image' in locals():
    whisp_image = whisp_image.addBands(custom_ee_image)
    print(f"Final image has {standard_bands + len(custom_bands)} bands ({standard_bands} + {len(custom_bands)} custom)")
else:
    print(f"Final image has {standard_bands} bands")

Run stats processing

In [ ]:
df_stats = whisp.whisp_formatted_stats_geojson_to_df(
    input_geojson_filepath=GEOJSON_EXAMPLE_FILEPATH,
    # external_id_column="user_id",  # optional - specify which input column/property to map to the external ID.
    national_codes=iso2_codes_list,  # optional - By default national datasets are not included unless specified here.
    # unit_type='percent',  # optional - to change unit type. Default is 'ha'.
    whisp_image=whisp_image,  # optional - defaults to standard whisp image if not provided
    custom_bands=custom_bands if USE_CUSTOM_BANDS else None,  # include custom bands in formatted output
    mode = "concurrent" # batch processing mode most useful for large datasets 
)


Display results

In [ ]:
df_stats

Calculate risk category

In [ ]:
# adds risk columns to end of dataframe
df_w_risk = whisp.whisp_risk(
    df=df_stats,
    national_codes=iso2_codes_list,
    custom_bands_info=custom_bands_info if USE_CUSTOM_BANDS else None,  # Add: missing custom bands
    # drop_unused_columns=True # set to True to remove stats columns not used for risk  
)

Display table with risk columns

In [ ]:
df_w_risk 

Export table to CSV

In [ ]:
# Generate timestamp 
timestamp = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M")
# Add a suffix to indicate notebook pathway (e.g., '_nb' for notebook)
output_filename = f"whisp_analysis_{timestamp}_nb.csv"

# Define the output folder (if running in Sepal change path to preferred folder) 
# e.g. out_directory = Path.home() / 'module_results/whisp/'
out_directory = Path.home() / 'downloads'
csv_output_file = out_directory / output_filename

# Save the CSV file
df_w_risk.to_csv(path_or_buf=csv_output_file, index=False)
print(f"Table with risk columns saved to: {csv_output_file}")

Export to GeoJSON (optional)

In [ ]:
# Define the output file path for GeoJSON
output_filename = f"whisp_analysis_{timestamp}_nb.geojson"
geojson_output_file = out_directory / output_filename

# Save the GeoJSON file
whisp.convert_df_to_geojson(df_w_risk, geojson_output_file)  # builds a geojson file containing Whisp columns. Uses the geometry column "geo" to create the spatial features.